## **Before you start**

Make a copy of this Colab by clicking on File > Save a Copy in Drive

After making a copy, add your student id, last name, and first name to the title.

In [ ]:
student_name = "Ricardo Sillas"
student_id = "88613474"
print("Identifying Information: ",student_name+","+student_id); # Don't change this line.

Identifying Information:  Ricardo Sillas,88613474


# Exercises Naive Bayes Classifier

## Naive Bayes - Example

Consider a dataset with 3 classes and 5 binary attributes. Let the arrays p_class and p_att_given_class be:

In [ ]:
import numpy as np

p_class = np.ones(3)/3
p_att_given_class = np.array([[0.72, 0.21, 0.89, 0.47, 0.64],[0.32, 0.82, 0.54, 0.82, 0.17],[0.76, 0.65, 0.74, 0.31, 0.75]])

print(p_class)
print()
print(p_att_given_class)


[0.33333333 0.33333333 0.33333333]

[[0.72 0.21 0.89 0.47 0.64]
 [0.32 0.82 0.54 0.82 0.17]
 [0.76 0.65 0.74 0.31 0.75]]


where p_class[i] represents the probability that an example belongs to class i and p_att_given_class[i,j] 
represents the probability that attribute j in an example of class i is equal to 1. 

What is the probability that for an example of class 2, attribute 0 is equal to 1?

In [ ]:
print(p_att_given_class[2,0])

0.76


What is the probability that for an example of class 1, attribute 4 is equal to 0?

In [ ]:
print(1-p_att_given_class[1,4])

0.83


How would the Naïve Bayes classifier classify example [1,1,1,0,0]? 

In [ ]:
def classify(x,pc,pac):
  p = pc 
  for i in range(x.shape[0]): #i = 3
    if x[i]==1:
      p = p* pac[:,i] 
    else:
      p = p*(1-pac[:,i])
  p = p/np.sum(p)
  # print(p)  Accuracy = 0.157
  return np.argmax(p)


In [ ]:
def classify_no_loops(x,pc,pac):
  p = x*pac + (1-x)*(1-pac)
  p = np.prod(p, axis=1)
  p = pc*p
  p = p/np.sum(p)
  # print(p)
  return np.argmax(p)

In [ ]:
test_example = np.array([1,1,1,0,0])

classify(test_example,p_class,p_att_given_class)

2

In [ ]:
test_example = np.array([1,1,1,0,0])

classify_no_loops(test_example, p_class, p_att_given_class)


2

## Exercise 1

Write a program to classify the MNIST dataset using the Naive Bayes classifer. 

Read the dataset

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.float32(x_train/255).reshape(x_train.shape[0],-1)
x_test = np.float32(x_test/255).reshape(x_test.shape[0],-1)

In [ ]:
for i in range(len(x_train)):
  if x_train != y_train:
    print(False)
    break
print(True)

False
True


<ipython-input-7-a511ac8a5cad>:2: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if x_train != y_train:


Create a new dataset using binary attributes as follows:

m = mean pixel value in x_train

x_train_binary[i,j]  =  1 if x_train[i,j] >m; otherwise x_train_binary[i,j] = 0

x_test_binary[i,j]  =  1 if x_test[i,j] >m; otherwise x_test_binary[i,j] = 0



In [ ]:
x_train_pixels = np.copy(x_train) 
x_test_pixels = np.copy(x_test)
m = np.mean(x_train)

x_train_pixels[x_train_pixels > m ] = 1
x_train_pixels[x_train_pixels <= m ] = 0

x_test_pixels[x_test_pixels > m ] = 1
x_test_pixels[x_test_pixels <= m ] = 0

Compute class probabilities (p_class) from training data, where p_class[i] represents the probability that an example belongs to class i. You can estimate this by dividing the number of instances in your training dataset that belong to class i over the number of training examples.

In [ ]:
p_class = np.zeros(10)

for i in range(p_class.size):
  p_class[i] = (y_train[y_train == i].size) / y_train.shape[0]

print(p_class)

[0.09871667 0.11236667 0.0993     0.10218333 0.09736667 0.09035
 0.09863333 0.10441667 0.09751667 0.09915   ]


Compute conditional probabilities (p_att_given_class) from training data, where p_att_given_class[i,j] represents the probability that attribute j in an example of class i is equal to 1. You can estimate this by dividing the number of times attribute j is equal to 1 in training instances of class j over the number of training instances of class i

In [ ]:
p_att_given_class = np.zeros((np.shape(p_class)[0],(np.shape(x_train)[1])))

for i in range(np.shape(p_att_given_class)[0]):
  indices = y_train == i
  p_att_given_class[i] = np.sum(x_train_pixels[indices], axis=0) / np.sum(indices) # axis=0 gets column sum

print(p_att_given_class)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Classify test data - print accuracy and confusion matrix (feel free to use sklearn to print the accuracy and confusion matrix)




In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Used to run classify_no_loops
x_predictions_one = np.zeros(np.shape(y_test)[0])
for x in range(np.shape(x_test_pixels)[0]):
  x_predictions_one[x] = classify_no_loops(x_test_pixels[x], p_class, p_att_given_class)

# Used to run classify
x_predictions_two = np.zeros(np.shape(y_test)[0])
for x in range(np.shape(x_test_pixels)[0]):
  x_predictions_two[x] = classify(x_test_pixels[x], p_class, p_att_given_class)

print("Accuracy with classify_no_loops method: {:.3f}".format(accuracy_score(y_test, x_predictions_one)))
print(confusion_matrix(y_test, x_predictions_one))

print("Accuracy with classify method: {:.3f}".format(accuracy_score(y_test, x_predictions_two)))
print(confusion_matrix(y_test, x_predictions_two))

Accuracy with classify_no_loops method: 0.846
[[ 893    0    4    5    1   33   22    1   21    0]
 [   0 1082   11    4    0   11    5    0   22    0]
 [  17   12  852   31   20    4   25   15   53    3]
 [   3   15   35  853    0   14    8   17   44   21]
 [   1    7    8    1  801    3   19    3   16  123]
 [  22    8    5  122   21  640   19    8   26   21]
 [  13   16   16    2   11   34  860    0    6    0]
 [   1   28   15    5   14    1    1  866   29   68]
 [  15   23   12   74   13   23   12    6  760   36]
 [  10   13    6    9   69    9    0   24   18  851]]
Accuracy with classify method: 0.846
[[ 893    0    4    5    1   33   22    1   21    0]
 [   0 1082   11    4    0   11    5    0   22    0]
 [  17   12  852   31   20    4   25   15   53    3]
 [   3   15   35  853    0   14    8   17   44   21]
 [   1    7    8    1  801    3   19    3   16  123]
 [  22    8    5  122   21  640   19    8   26   21]
 [  13   16   16    2   11   34  860    0    6    0]
 [   1   28   1

## Submission Instructions

1. File > Download .ipynb
2. Go to Blackboard, find the submission page, and upload the .ipynb file you just downloaded.